In [1]:
import torch
from segment_anything import SamPredictor, sam_model_registry

sam_path = '../weights/sam_vit_b_01ec64.pth'
sam = sam_model_registry["vit_b"](checkpoint=sam_path)
device = torch.device('cuda:0')
sam.to(device)
predictor = SamPredictor(sam)

In [7]:
import numpy as np
import os
import sys

current_folder = os.getcwd()
parent_folder = os.path.dirname(current_folder)
sys.path.extend([parent_folder, f'{parent_folder}/segnext'])

from tqdm import tqdm
from segnext.isegm.data.datasets import DavisDataset, HQSeg44kDataset
from segnext.isegm.inference.clicker import Clicker
from segnext.isegm.inference.utils import get_iou


def get_points_nd(clicks_list):

    points, labels = [], []
    for click in clicks_list:
        h, w = click.coords_and_indx[:2]
        points.append([w, h])
        labels.append(int(click.is_positive))
    return np.array(points), np.array(labels)


def eval(dataset):
    max_clicks = 20
    pred_thr = 0.5
    results = []
    for index in tqdm(range(len(dataset)), leave=False):
        sample = dataset.get_sample(index)
        for object_id in sample.objects_ids:
            image = sample.image
            gt_mask = sample.gt_mask(object_id)
            pred_mask = np.zeros_like(gt_mask)
            pred_logits = None

            clicker = Clicker(gt_mask=gt_mask)
            predictor.set_image(image)

            ious = []
            for _ in range(max_clicks):
                clicker.make_next_click(pred_mask)
                clicks_list = clicker.get_clicks()
                points_nd, labels_nd = get_points_nd(clicks_list)

                preds, scores, pred_logits = predictor.predict(
                    points_nd, labels_nd, mask_input=pred_logits)
                
                max_score_idx = np.argmax(scores)
                pred_mask = preds[max_score_idx] > pred_thr
                pred_logits = pred_logits[[max_score_idx]]

                iou = get_iou(gt_mask, pred_mask)
                ious.append(iou)
            results.append(ious)
    return results


In [8]:
# Evaluate on DAVIS
DAVIS_PATH="../data/DAVIS345"
DAVIS = DavisDataset(DAVIS_PATH)
results = eval(DAVIS)

In [9]:
# Calculate metrics for DAVIS
from segnext.isegm.inference import utils


def compute_noc_metric(all_ious, iou_thrs, max_clicks=20):
    def _get_noc(iou_arr, iou_thr):
        vals = iou_arr >= iou_thr
        return np.argmax(vals) + 1 if np.any(vals) else max_clicks

    noc_list = []
    noc_list_std = []
    over_max_list = []
    for iou_thr in iou_thrs:
        scores_arr = np.array([_get_noc(iou_arr, iou_thr)
                               for iou_arr in all_ious], dtype=int)

        score = scores_arr.mean()
        score_std = scores_arr.std()
        over_max = (scores_arr == max_clicks).sum()

        noc_list.append(score)
        noc_list_std.append(score_std)
        over_max_list.append(over_max)

    return noc_list, noc_list_std, over_max_list


noc_list, noc_list_std, over_max_list = compute_noc_metric(
    results, iou_thrs=np.array([0.85, 0.90, 0.95]), max_clicks=20)

mIoU = np.array(results).mean(axis=0)

print('NoC@90 = ', noc_list[1])
print('NoC@95 = ', noc_list[2])
print('NoF@95 = ', over_max_list[2])
print('5-mIoU = ', mIoU[4])


NoC@90 =  5.147826086956521
NoC@95 =  10.791304347826086
NoF@95 =  151
5-mIoU =  0.9094677801517471
